In [50]:
import cv2
import numpy as np
import imgaug as ia
from imgaug import augmenters as iaa
import imageio
import imutils
import os

In [2]:
folder = './photos'

# Bounding box

In [71]:
def get_coord(path: str, label: int) -> list:
    """
    Функция выделяет из маски изображения  нормализированные координаты, выделенных объектов
        path: путь к маски изображения
        label: класс объекта
        
        return: список кортежей вида (label, x_center, y_center, w, h)
    """
    features = list()
    mask = cv2.imread(path)
    h_v, w_v = mask.shape[0], mask.shape[1]
    gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]
    contours, _ = cv2.findContours(thresh.copy(), 1, 1)

    for i in contours:
        area = cv2.contourArea(i)
        x,y,w,h = cv2.boundingRect(i)
        x = (x+int(w/2))/w_v
        y = (y+int(h/2))/h_v
        w = w/w_v
        h = h/h_v
        rect = cv2.minAreaRect(i) # basically you can feed this rect into your classifier
        if label == 0:
            if area > 1000:
                features.append((label,x,y,w,h))
        elif label == 1:
            if area > 1000:
                features.append((label,x,y,w,h))
        elif label == 2:
            if area > 1000:
                features.append((label,x,y,w,h))
        elif label == 3:
            if area < 10000:
                features.append((label,x,y,w,h))
    
    
    return features

In [72]:
def save_coord(path: str) -> list:
    """
    Функция разделяет маски на классы и записывает необходимую информацию в файл .txt
        классы объектов: {'metall':0, 'net':1, 'plastic':2, 'wood':3}
        
        path: путь к изображению без префмйка '_image'
        
        return: список кортежей к данными по обнаруженным объектам на изображении
    """
    #

    massive = list()
    folder_result = path + '_image'
    
    if os.path.exists(path + '_net.png'):
        massive.extend(get_coord(path + '_net.png', 1))
    if os.path.exists(path + '_wood.png'):
        massive.extend(get_coord(path + '_wood.png', 3))
    if os.path.exists(path + '_plastic.png'):
        massive.extend(get_coord(path + '_plastic.png', 2))
    if os.path.exists(path +  '_metall.png'):
        massive.extend(get_coord(path + '_metall.png', 0))
    
    with open(f'{folder_result}.txt', 'w') as f:
        for value in massive:
            f.write(f'{value[0]} {value[1]} {value[2]} {value[3]} {value[4]}\n')
        
    return massive

In [73]:
for i in range(0,21):
    if i < 10
        save_coord(folder + '/0' + str(i))
        
    else:
        save_coord(folder + '/' + str(i))

1 ./photes/00_image
1 ./photes/00_image
1 ./photes/00_image
1 ./photes/00_image
1 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
3 ./photes/00_image
1 ./photes/01_image
1 ./photes/01_image
1 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
3 ./photes/01_image
2 ./photes/01_image
2 ./photes/01_image
2 ./photes/01_image
2 ./photes/01_image
3 ./photes/02_image
3 ./photes/02_image


# Augmentation

In [78]:
def augmentations(path, path_save, num_img) -> None:
    """
    Функция агументации оригинальных изображений
        
         path: путь к изображению без префикса '_image.JPG'
         path_save: путь сохранения аугментированного изображения
         num_img: порядковый номер изображения
    """
    
    image = imageio.imread(path + '_image.JPG')


    cutout = iaa.Cutout(fill_mode="constant", fill_per_channel=1)
    dropout = iaa.CoarseDropout(0.01, size_percent=0.02, per_channel=0.01)# - norm
    invert = iaa.Invert(0.25, per_channel=1)
    dropout_chan = iaa.CoarseDropout(p=0.2, per_channel=True)
    mult_black = iaa.Multiply((0.5, 1.5), per_channel=1)

    image_drop = dropout(image=image)
    image_cut = cutout(image=image)
    images_inv = invert(image=image)
    image_drop_chan = dropout_chan(image=image)
    image_mult = mult_black(image=image)
    
    cv2.imwrite(path_save + '/' + 'droppixcel/' + num_img + '.JPG', image_drop)
    cv2.imwrite(path_save + '/' + 'cutout/' + num_img + '.JPG', image_cut)
    cv2.imwrite(path_save + '/' + 'inv/' + num_img + '.JPG', images_inv)
    cv2.imwrite(path_save + '/' + 'drop_chan/' + num_img + '.JPG', image_drop_chan)
    cv2.imwrite(path_save + '/' + 'mult/' + num_img + '.JPG', image_mult)

In [79]:
for i in range(0,52):
    if i < 10:
        augmentations(folder + '/0' + str(i), folder, '0' + str(i))
        
    else:
        augmentations(folder + '/' + str(i), folder, '0' + str(i))